In [ ]:
#author: Mohan Kancherla
#CSCI 5702, Big Data Mining Assignment2

#importing numpy, sparkcontext,distance and matlab plot
import numpy as np
from pyspark import SparkContext
from scipy.spatial import distance
import matplotlib.pyplot as plt

#opening spark session
sc = SparkContext()
# assigning default variables : iterations, datapath , centriod path and bool value
Iterations = 20
dataset_path = r"C:\Users\mohan\BigDataMining\data.txt"
centroids_path = r"C:\Users\mohan\BigDataMining\c1.txt"
euclidean_distance = True
# Reading data to numpy array as float values
data1= np.loadtxt(dataset_path,dtype=float)
# Changing the data into RDD
data_rdd = sc.parallelize(data1)
data = data_rdd.map(lambda x : np.array(x.tolist()))
# Reading centriods data into numpy array
centroids_data = np.loadtxt(centroids_path)
# definition of kmeans function with default parameters
def kmeans(data,centroids, max_iterations, euclidean):
    # assigning costs lists as empty lists 
    totalcost_euc=[]
    totalcost_man=[]
    # chether whether the bool passed is true or false
    if(euclidean is True):
        distType = "Euclidean"
    else:
        distType = "Manhattan"
    # printing out iteration and distance type
    print("Max Iterations : {}, distance measure : {}".format(max_iterations, distType))
    # looping through max iterations
    for i in range(1, max_iterations+1):
        # finding clusters using euclidean distance
        if euclidean == True:
            # finding distance for each data point with centriods
            dist = data.map(lambda x: (x, np.array([distance.euclidean(x, z) for z in centroids])))
            # mapping with lowest distance centriod index
            dist2 = dist.map(lambda x: (np.argmin(x[1]),x[0]))
            # group and sort by key value
            dist3 = dist2.groupByKey().sortByKey().mapValues(list)
            # finding cost incurred for the above iteration
            Cost = data.map(lambda x: np.array([(distance.euclidean(x, z))**2 for z in centroids]))
            Cost2 = Cost.map(lambda x: np.min(x))
            #print(Cost2.collect())
            # finding overallcost incurred with the mentioned formula in pdf
            Cost3 = sum(Cost2.collect())
            # printing out total cost values
            print("Iteration {}: {}".format(i, Cost3))
            # appending into list
            totalcost_euc.append(Cost3)
            # calculating new centriods for the found clusters
            new_centroids = dist3.map(lambda x: np.average(x[1], axis=0))
        else:
            # using manhattan distance, calculating the distance between centriod and each data point
            dist = data.map(lambda x: (x, np.array([distance.cityblock(x, z) for z in centroids])))
            dist2 = dist.map(lambda x: (np.argmin(x[1]),x[0]))
            # minimum values are grouped and sorted by key values
            dist3 = dist2.groupByKey().sortByKey().mapValues(list)
            # finding the cost incuured using manhattan distance
            Cost = data.map(lambda x: np.array([(distance.cityblock(x, z)) for z in centroids]))
            Cost2 = Cost.map(lambda x: np.min(x))
            # total cost incurred
            totalcost = sum(Cost2.collect())
            print("Iteration {}: {}".format(i, totalcost))
            totalcost_man.append(totalcost)
            # calculating new centriods with the found clusters
            new_centroids = dist3.map(lambda x: np.average(x[1], axis=0))
            
        # new centriods is passed to the for loop to recompute
        centroids = np.array(new_centroids.collect())
    # if we have euclidean values, it will plot the graph or else it will plot with manhattan distance
    if(len(totalcost_euc)>0):
        plt.plot([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], totalcost_euc)
        plt.xlabel('Iteration')
        plt.ylabel('Total cost using Eculidean distance')
        plt.title('Iteration vs Total cost incurred')
        plt.show()
    elif(len(totalcost_man)>0):
        plt.plot([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], totalcost_man)
        plt.xlabel('Iteration')
        plt.ylabel('Total cost using manhattan distance')
        plt.title('Iteration vs Total cost incurred')
        plt.show()
    else:
        print("Graph is not plotted")

# calling the function with true value as euclidean distance
kmeans(data,centroids_data, Iterations,euclidean_distance)
# calling the kmeans function with false value as manhattan distance
kmeans(data,centroids_data, Iterations,False)

# finally closing the spark session
sc.stop()